In [13]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

import math

# Recuperamos la lista de "hiperparámetros" con diferencias
# estadísticamnete significativas
with open("test7", "rb") as f:
    list_df_signif = pickle.load(f)

En esta lista tenemos tuplas con las matrices que hay por cada preproceso, trozo y canal. Estas tuplas tienen la forma:
* DataFrame con la matriz correspondiente
* int que indica el tipo de preproceso
* int que indica el trozo de la grabación
* int que indica el canal

### Clasificacón KNN

In [14]:
grid_knn = {'n_neighbors': [3,4,5],
        'weights':['uniform', 'distance'],
        'algorithm': ['auto'],
        'metric': ['minkowski', 'mahalanobis','euclidean', 'cosine']
        }

In [ ]:
list_results_knn = []
cv_results_knn = []
for data in list_df_signif[:10]:
    (df_data, prep, troz, chan) = data
    print('prep:',prep,'troz:', troz,'chan:', chan)
    data_arr = df_data.iloc[:,:-1].to_numpy()
    label_arr = df_data['Label'].to_numpy()
    knn = KNeighborsClassifier()
    clf = GridSearchCV(estimator=knn, param_grid=grid_knn, cv=5, return_train_score=True,
                               verbose=2)
    clf.fit(data_arr, label_arr)
    list_results_knn.append(clf)
    cv_results_knn.append((clf.cv_results_, prep, troz, chan))
#128min

In [16]:
def conversor_num_to_chan(num):
    if num == 0:
        a = 'Fp1'
    elif num == 1:
        a = 'Fp2'
    elif num == 2:
        a = 'F3'
    elif num == 3:
        a = 'F4'
    elif num == 4:
        a = 'C3'
    elif num == 5:
        a = 'C4'
    elif num == 6:
        a = 'P3'
    elif num == 7:
        a = 'P4'
    elif num == 8:
        a = 'O1'
    elif num == 9:
        a = 'O2'
    elif num == 10:
        a = 'F7'
    elif num == 11:
        a = 'F8'
    elif num == 12:
        a = 'T7'
    elif num == 13:
        a = 'T8'
    elif num == 14:
        a = 'P7'
    elif num == 15:
        a = 'P8'
    elif num == 16:
        a = 'Fz'
    elif num == 17:
        a = 'Cz'
    elif num == 18:
        a = 'Pz'
    else:
        print('error de canales')
    return a

In [17]:
(dic_res,prep,troz,chan) = cv_results_knn[0]
df = pd.DataFrame(dic_res)
ind_max = df['mean_test_score'].idxmax()
df.iloc[ind_max,:]

mean_fit_time                                                       0.0
std_fit_time                                                        0.0
mean_score_time                                                0.001109
std_score_time                                                 0.000859
param_algorithm                                                    auto
param_metric                                                  minkowski
param_n_neighbors                                                     4
param_weights                                                   uniform
params                {'algorithm': 'auto', 'metric': 'minkowski', '...
split0_test_score                                                  0.64
split1_test_score                                                 0.875
split2_test_score                                              0.583333
split3_test_score                                              0.708333
split4_test_score                                              0

In [18]:
list_accuracies_knn = []
for (dic_res,prep,troz,chan) in cv_results_knn:
    df = pd.DataFrame(dic_res)
    ind_max = df['mean_test_score'].idxmax()
    acc = df.iloc[ind_max]['mean_test_score']
    std_error = df.iloc[ind_max]['std_test_score']
    param_n_neighbors = df.iloc[ind_max]['param_n_neighbors']
    param_weights = df.iloc[ind_max]['param_weights']
    param_metric = df.iloc[ind_max]['param_metric']
    lab_chan = conversor_num_to_chan(chan)
    list_accuracies_knn.append((prep, troz, lab_chan, param_n_neighbors,\
                                param_weights, param_metric, acc, std_error))

In [19]:
list_accuracies_knn.sort(key=lambda x: x[6], reverse=True)
df_results_knn = pd.DataFrame(list_accuracies_knn)
df_results_knn.columns = ['preproceso', 'trozo', 'canal', \
                          'n_neighbors', 'weights', 'metric', 'accuracy',\
                          'std_error']

In [20]:
df_results_knn

,preproceso,trozo,canal,n_neighbors,weights,metric,accuracy,std_error
0,0,0,P3,5,distance,cosine,0.728000,0.086554
1,0,0,Fp1,4,uniform,minkowski,0.703000,0.097908
2,0,0,C3,4,uniform,minkowski,0.678000,0.079144
3,0,0,F3,3,uniform,cosine,0.677667,0.071574
4,0,0,Fp2,5,uniform,cosine,0.677000,0.078418
5,0,0,F4,4,uniform,cosine,0.669667,0.062870
6,0,0,O2,5,uniform,cosine,0.636000,0.097528
7,0,0,O1,4,uniform,minkowski,0.629333,0.087513
8,0,0,P4,4,uniform,cosine,0.621000,0.089490
9,0,0,C4,5,uniform,cosine,0.604000,0.067393


In [5]:
# list_best_params_knn = []
# for best_p in list_results_knn:
#     list_best_params_knn.append(best_p.best_params_)

In [6]:
# list_accuracies_knn.sort(reverse=True)

In [7]:
# def conversor_num_mat(num):
#     chan = num%19
#     n_troz = math.floor(num/19)
#     troz = n_troz%35
#     n_prep = math.floor(n_troz/35)
#     prep = n_prep%4
#     return prep, troz, chan

In [8]:
# list_good_res_knn = []
# for tup in list_accuracies_knn:
#     acc = tup[0]
#     pos = tup[2]
#     prep, troz, chan = conversor_num_mat(pos)
#     dic = list_results_knn[pos].best_params_
#     dic['preproceso'] = prep
#     dic['trozo'] = troz
#     dic['channel'] = chan
#     dic['accuracy'] = acc
#     list_good_res_knn.append(dic)
# df_results_knn = pd.DataFrame(list_good_res_knn)

In [10]:
# a = list(df_results_knn['channel'])
# b = [conversor_num_to_chan(i) for i in a]
# df_results_knn['channel'] = b

In [11]:
df_results_knn.head(50)
# Nuevo

,algorithm,metric,n_neighbors,weights,preproceso,trozo,channel,accuracy
0,auto,cosine,4,distance,1,13,Fz,0.811000
1,auto,cosine,4,uniform,0,29,Fz,0.810000
2,auto,minkowski,3,uniform,0,0,Pz,0.802667
3,auto,cosine,4,uniform,0,28,Fz,0.802000
4,auto,minkowski,5,distance,0,29,P3,0.794000
5,auto,minkowski,4,uniform,0,25,P3,0.786333
6,auto,minkowski,5,uniform,0,28,P7,0.785667
7,auto,minkowski,4,uniform,0,28,P3,0.785667
8,auto,minkowski,4,uniform,0,13,P3,0.785667
9,auto,cosine,5,uniform,0,18,Fz,0.784667


In [ ]:
df_results_knn.head(50)

In [12]:
with open("results_DES_knn", "wb") as f:   #Pickling
    pickle.dump(df_results_knn, f)

Estos resultados están guardados en `results_DES_knn`